In [ ]:
# SETUP destination directory
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
# SETUP environement
!pip install transformers==4.36.0
!pip install git+https://github.com/huggingface/accelerate.git -q -U
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/peft.git -q -U
!pip install --no-cache-dir sentencepiece
!pip install -q datasets einops wandb trl

import sentencepiece, torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl (102.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# IMPORT DATASET from huggingFace
%%capture
dataset_name = "Maxime62/JuniaLLM"
dataset = load_dataset(dataset_name)['train']

In [ ]:
# We will use the Vigogne model
# Vigogne is a collection of powerful French large language models (LLMs) that are open-source and designed for instruction-following
model_name = "bofenghuang/vigogne-2-7b-instruct"

# Fine-tuned model name
new_model = "vigogne-2-7b-Junia"

# !!! FineTuning parameters have been optimized with Unsloth !!!

In [ ]:
# Creating new tokens for our JUNIA LLM
new_tokens = [
"ISEN", "ISA", "HEI", "JUNIA"
]

In [ ]:
# LOADING TOKENIZER
max_seq_length = 2048

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    max_seq_length = max_seq_length,
    use_fast=False,
    truncation=True
)
tokenizer.pad_token = tokenizer.bos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# LOADING MODEL
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # dataset load is done in 4-bit
    bnb_4bit_quant_type="nf4",# The "nf4" value suggests that the model is using "narrow full" 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16, #computation are done in 16-bit fp
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/175 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
# Add new tokens to the tokenizer's vocabulary
tokenizer.add_tokens(new_tokens)

# Resize the token embedding matrix to match the new vocabulary size
model.resize_token_embeddings(len(tokenizer))

Embedding(32005, 4096)

In [ ]:
# Function is designed to format the elements of a dataset
def formatting_prompts_func(examples):
    output_text = [] # will hold the formatted text
    for i in range(len(examples)):
        instruction = examples["instruction"][i]
        response = examples["output"][i]

        text = f"""<s>Ci-dessous se trouve une instruction qui décrit une demande d'un étudiant de chez Junia. Rédigez une réponse qui répond de manière précise à la demande.

### Instruction:
{instruction}

### Response:
{response}</s>"""

        output_text.append(text)
    return output_text # return the final output of the formated text

In [ ]:
# LoRA
# Setting up hyperparameters for LoraConfig
lora_r = 64
lora_alpha = 16
lora_dropout = 0

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    task_type="CAUSAL_LM"
)

In [ ]:
# Setting up the training configuration for the model

training_arguments = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    max_steps = 60,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "./Fine Tuning"
)

In [ ]:
# Setting up the trainer for fine-tuning
%%capture
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    max_seq_length = max_seq_length,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func, # Formatting the dataset with the function defined above
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args=training_arguments,
)

In [ ]:
# Training the model
trainer.train()

# Save trained model
trainer.model.save_pretrained("Output")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.261500
2,1.232500
3,1.206600
4,1.140500
5,1.061200
6,0.910600
7,0.793100
8,0.663400
9,0.557100
10,0.468000


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:
# Merge the model and LoRa
peftModel = PeftModel.from_pretrained(model, "Output")

In [ ]:
# Upload the model on HuggingFace
!huggingface-cli login

peftModel.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


adapter_model.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Maxime62/vigogne-2-7b-Junia/commit/5ed27932c0ae4531f8803c1fd917055bce98e09e', commit_message='Upload tokenizer', commit_description='', oid='5ed27932c0ae4531f8803c1fd917055bce98e09e', pr_url=None, pr_revision=None, pr_num=None)